# Exercise 3.1: Demonstrating Durability — Solution

## Setup

Before doing the exercise, you need to:

- Install necessary dependencies
- Create your `.env` file and supply your API key
- Load the environment variables
- Download and start a local Temporal Service

In [1]:
%pip install --quiet temporalio openai-agents httpx nest-asyncio pytz

# Import all required modules
import asyncio
import httpx
from datetime import timedelta, datetime
import pytz
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.worker import Worker
from temporalio.contrib import openai_agents
from temporalio.contrib.openai_agents import OpenAIAgentsPlugin, ModelActivityParameters
from temporalio.worker.workflow_sandbox import SandboxedWorkflowRunner, SandboxRestrictions
from agents import Agent, Runner

print("✅ All imports successful")


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ All imports successful


## Component 1: `activities.py`

Define the weather activity that fetches alerts from the National Weather Service API.

**Key points:**
- Returns a `dict` (not formatted string) so the LLM can interpret the data
- Activity name explicitly set with `@activity.defn(name="get_weather")`
- Simple error handling - just `raise_for_status()` and let Temporal retry
- No console logging or attempt tracking needed

In [2]:
from temporalio.exceptions import ApplicationError

@activity.defn(name="get_weather")  # Register this function as a Temporal activity
async def get_weather_error(state: str) -> dict:  # Accept 2-letter state code, return structured data
    """Fetch active NWS alerts for a 2-letter US state code w/ error (e.g., 'CA')."""
    # This simulates a temporary failure - maybe a database is down,
    # or an API is temporarily unavailable
    raise ApplicationError("Simulated failure: Summary service temporarily unavailable")

print("✅ Activity 'get_weather' defined")

✅ Activity 'get_weather' defined


In [4]:
# Define which task queue this workflow will use for communication
TASK_QUEUE = "agents-sdk-error"

@workflow.defn(sandboxed=False)  # Disable sandbox for Jupyter compatibility
class ErrorWorkflow:  # Define workflow class for orchestrating the agent
    @workflow.run  # Mark this method as the workflow entry point
    async def run(self, user_query: str) -> str:  # Accept user query, return agent response
        # Create the Agent inside the workflow with tools and instructions
        agent = Agent(
            name="Weather Assistant",  # Name shown in OpenAI traces
            # Define agent's role and behavior when handling user queries
            instructions=(
                "You are a helpful assistant that explains current weather alerts for U.S. states. "
                "When the user asks about alerts, call the get_weather tool, then summarize results."
            ),
            # Provide list of tools the agent can use
            tools=[
                # Convert Temporal activity to Agent tool for durable execution
                openai_agents.workflow.activity_as_tool(
                    # Update the activity function to the error version
                    get_weather_error,
                    start_to_close_timeout=timedelta(seconds=10),  # Max time for activity execution
                )
            ],
        )
        
        # Run the agent with user query - tool calls execute as Temporal activities
        result = await Runner().run(agent, user_query)
        
        # Safely extract final output from result object (handles different result types)
        return getattr(result, "final_output", str(result))

print(f"✅ Workflow 'ErrorWorkflow' defined (task queue: {TASK_QUEUE})")

✅ Workflow 'ErrorWorkflow' defined (task queue: agents-sdk-error)


## Component 3: `worker.py`

Run the worker that polls for and executes workflow/activity tasks.

**Key points:**
- Connects to Temporal with `OpenAIAgentsPlugin` (required)
- Configures `ModelActivityParameters` for AI model timeout settings
- Uses `SandboxedWorkflowRunner` with passthrough for `httpx`
- Runs in background as an async task in Jupyter

In [ ]:
async def run_worker():  # Define async function to start and run the worker
    """Start a Temporal worker that listens for workflow and activity tasks."""
    # Connect to local Temporal server with OpenAI Agents SDK plugin
    client = await Client.connect(
        "localhost:7233",  # Temporal server address
        # Register plugins to extend Temporal's functionality
        plugins=[
            # Plugin that enables OpenAI Agents SDK integration with Temporal
            OpenAIAgentsPlugin(
                # Configure timeout settings for AI model inference activities
                model_params=ModelActivityParameters(
                    start_to_close_timeout=timedelta(seconds=30)  # Max time for LLM calls
                )
            )
        ],
    )

    # Create worker that polls the task queue for work
    worker = Worker(
        client,  # Use the connected Temporal client
        task_queue=TASK_QUEUE,  # Which queue to poll for tasks
        # Udpate to the ErrorWorkflow
        workflows=[ErrorWorkflow],
        # Use the activity that has an intentional error
        activities=[get_weather_error],  # List of activities this worker can execute
    )
    
    print(f"✅ Worker started on task queue: {TASK_QUEUE}")
    print("   Listening for workflow and activity tasks...")
    # Start polling and executing tasks (blocks until stopped)
    await worker.run()

# Apply nest_asyncio to allow nested event loops in Jupyter
import nest_asyncio

nest_asyncio.apply()
# Create background task to run worker without blocking the notebook
worker_task = asyncio.create_task(run_worker())
print("🔄 Worker running in background")

🔄 Worker running in background


✅ Worker started on task queue: agents-sdk-error
   Listening for workflow and activity tasks...


Completing activity as failed ({'activity_id': '2', 'activity_type': 'get_weather', 'attempt': 1, 'namespace': 'default', 'task_queue': 'agents-sdk-error', 'workflow_id': 'weather-wed-oct-22-121217est', 'workflow_run_id': '019a0cb1-4c7a-7080-9ea8-c07d5366fc32', 'workflow_type': 'ErrorWorkflow'})
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_activity.py", line 297, in _handle_start_activity_task
    result = await self._execute_activity(start, running_activity, task_token)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_activity.py", line 610, in _execute_activity
    return await impl.execute_activity(input)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/contrib/openai_agents/_trace_interceptor.py", line 284, in execute_activity
    return await self.

## Component 4: `starter.py`

Running a Temporal Workflow.

### Executing the Workflow
* Temporal Workflows are executed indirectly
* Request execution from the Temporal Service
* You do this with the Temporal Client

In [6]:
async def run_solution():  # Define async function to execute the workflow
    """Execute the weather agent workflow."""
    # Define the user query to send to the agent
    query = "What weather alerts are active in CA?"
    
    # Generate workflow ID with EST timestamp for human-readable tracking
    est = pytz.timezone('US/Eastern')  # Create EST timezone object
    now = datetime.now(est)  # Get current time in EST
    # Format timestamp as readable string with day-month-date-time pattern
    workflow_id = f"error-{now.strftime('%a-%b-%d-%I%M%S').lower()}est"
    
    # Connect to Temporal server with OpenAI Agents SDK plugin
    client = await Client.connect(
        "localhost:7233",  # Temporal server address
        plugins=[OpenAIAgentsPlugin()]  # Enable OpenAI Agents integration
    )
    
    print(f"🚀 Starting workflow: {workflow_id}")
    print(f"📝 Query: {query}\n")
    
    # Start the workflow (non-blocking) and get handle for tracking
    handle = await client.start_workflow(
        ErrorWorkflow.run,  # Update to the ErrorWorkflow
        query,  # User query parameter passed to workflow
        id=workflow_id,  # Unique workflow ID for tracking in Temporal UI
        task_queue=TASK_QUEUE  # Queue where worker will pick up this workflow
    )
    
    print(f"✅ Workflow started: {handle.id}")
    # Print Temporal UI link for observing workflow execution
    print(f"🔗 View in Temporal UI: http://localhost:8233/namespaces/default/workflows/{workflow_id}\n")
    print("⏳ Waiting for agent response...\n")
    
    # Wait for workflow to complete and get result (blocking)
    result = await handle.result()
    
    print("=" * 60)
    print("🤖 Agent Response:")
    print("=" * 60)
    print(result)
    print("=" * 60)

# Run the solution with Jupyter-specific async handling
try:
    # Try to get existing event loop (Jupyter has one running)
    loop = asyncio.get_running_loop()
    # Execute in existing loop
    await run_solution()
except RuntimeError:
    # If no loop exists, create new one and run
    asyncio.run(run_solution())

🚀 Starting workflow: weather-wed-oct-22-121217est
📝 Query: What weather alerts are active in CA?

✅ Workflow started: weather-wed-oct-22-121217est
🔗 View in Temporal UI: http://localhost:8233/namespaces/default/workflows/weather-wed-oct-22-121217est

⏳ Waiting for agent response...



CancelledError: 